In [1]:
import numpy as np
import pandas as pd
import itertools
import random

from sklearn.model_selection import train_test_split

from MIL.mil_utils import *
from evaluation import *

In [2]:
# Dataset info
env = 'CCHS'
id_feat = 'VisitIdentifier'
sel_feats = ['SystolicBP', 'MAP', 'RespiratoryRate', 'PulseOx', 'HeartRate', 'Temperature', 
             'WBC', 'BiliRubin', 'BUN', 'Lactate', 'Creatinine', 'Platelet', 'Bands', 'FIO2']
action_dim = 2 

max_sel_feats = ['max_' + i for i in sel_feats]
min_sel_feats = ['min_' + i for i in sel_feats]
all_feats = sel_feats + max_sel_feats + min_sel_feats
state_dim = len(all_feats) 

In [3]:
# Load the data 
df = pd.read_csv('../baseline/example/cchs_sample_data.csv')
vid_list = sorted(np.unique(df.VisitIdentifier))

# Get the list of dataframes
df_list = []
vid_list = sorted(np.unique(df.VisitIdentifier))
for vid in vid_list: 
    df_list.append(df.loc[df.VisitIdentifier == vid])

In [4]:
seed_rp_results = []
for rp_idx in range(1): 
    # ------------------------------------------------------------------------------------
    # Get the training and testing index
    tr_idx, te_idx = train_test_split(np.arange(len(df_list)), test_size=0.2, random_state=rp_idx)
    
    # Slice the training & testing data
    tr_df = [df_list[i] for i in tr_idx]
    te_df = [df_list[i] for i in te_idx]

    # ------------------------------------------------------------------------------------
    # Get the expert trajectories
    tr_states = list(itertools.chain.from_iterable([[row for row in np.array(tr_df[i][all_feats])] for i in range(len(tr_df))]))
    tr_actions = list(itertools.chain.from_iterable([[row for row in np.array(tr_df[i]['Action'])] for i in range(len(tr_df))]))

    # Augment the training data
    pos_idx = [i for i in range(len(tr_actions)) if tr_actions[i] == 1]
    aug_tr_states = [tr_states[i] for i in pos_idx] * 2
    aug_tr_actions = [tr_actions[i] for i in pos_idx] * 2
    tr_states.extend(aug_tr_states)
    tr_actions.extend(aug_tr_actions)
    
    suf_idx = np.arange(len(tr_states))
    random.seed(42)
    random.shuffle(suf_idx)
    
    tr_states = [tr_states[i] for i in suf_idx]
    tr_actions = [tr_actions[i] for i in suf_idx]
    tr_actions = [np.array([.8, 0]) if x == 0 else np.array([0, .8]) for x in tr_actions]
    expert_data = [(s,a) for s,a in zip(tr_states, tr_actions)]

    # ------------------------------------------------------------------------------------
    # Training the MIL model
    num_intentions = 3 # Latent variable dimension (context)
    epochs = 20 #1000
    batch_size = 64
    
    # Create models
    latent_intention_network = LatentIntentionNetwork(num_intentions, num_intentions)
    generator = Generator(state_dim, action_dim, latent_intention_network)
    discriminator = Discriminator(state_dim, action_dim) #, num_intentions)
    
    # Optimizers
    optimizer_g = tf.keras.optimizers.Adam(learning_rate=1e-3)
    optimizer_d = tf.keras.optimizers.Adam(learning_rate=1e-4)
    
    # Train the InfoGAIL model
    generator, discriminator = train_info_gail(expert_data, generator, discriminator, optimizer_g, optimizer_d, 
                                               num_intentions, epochs=epochs, batch_size=batch_size, print_interval=2)

    # ------------------------------------------------------------------------------------
    # Testing the MIL model
    # Get the testing trajectories
    te_states = list(itertools.chain.from_iterable([[row for row in np.array(te_df[i][all_feats])] for i in range(len(te_df))]))
    te_actions = list(itertools.chain.from_iterable([[row for row in np.array(te_df[i]['Action'])] for i in range(len(te_df))]))
    te_actions = [np.array([.8, 0]) if x == 0 else np.array([0, .8]) for x in te_actions]
    test_data = [(s,a) for s,a in zip(te_states, te_actions)]

    # Evaluate the model
    true_act, pred_act, pred_prob = [], [], []
    avg_patterns = 'binary'
    
    for sa_idx in range(len(te_states)):
        z = sample_latent_variable(num_intentions)  # Sample a latent variable for each state
        latent_intention_probs = latent_intention_network(z)  # Get intention probabilities
        generated_action = generator(te_states[sa_idx], z, latent_intention_probs)  # Get the action from the generator
        
        pred_prob.append(tf.nn.softmax(generated_action).numpy()[0])
        pred_act.append(np.argmax(generated_action.numpy()[0]))
        true_act.append(np.argmax(te_actions[sa_idx]))
    
    seed_rp_results.append(overall_eval(true_act, pred_act, pred_prob, 2, avg_patterns=avg_patterns))

C:\Users\XiYang\miniforge3\envs\themes\lib\site-packages\keras\src\optimizers\base_optimizer.py:774: UserWarning: Gradients do not exist for variables ['latent_intention_network/dense/kernel', 'latent_intention_network/dense/bias', 'latent_intention_network/dense_1/kernel', 'latent_intention_network/dense_1/bias', 'latent_intention_network/dense_2/kernel', 'latent_intention_network/dense_2/bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


Epoch 0: Generator Loss = 0.00897879246622324, Discriminator Loss = 0.02106037177145481
Epoch 2: Generator Loss = 0.008957655169069767, Discriminator Loss = 0.019060995429754257
Epoch 4: Generator Loss = 0.008949297480285168, Discriminator Loss = 0.01636641100049019
Epoch 6: Generator Loss = 0.008946222253143787, Discriminator Loss = 0.012730655260384083
Epoch 8: Generator Loss = 0.008920262567698956, Discriminator Loss = 0.008750165812671185
Epoch 10: Generator Loss = 0.008903169073164463, Discriminator Loss = 0.005507076159119606
Epoch 12: Generator Loss = 0.008886197581887245, Discriminator Loss = 0.003309172810986638
Epoch 14: Generator Loss = 0.008869818411767483, Discriminator Loss = 0.002018131548538804
Epoch 16: Generator Loss = 0.008858912624418736, Discriminator Loss = 0.001273058820515871
Epoch 18: Generator Loss = 0.008824581280350685, Discriminator Loss = 0.0008439882658421993
Performance Measurements:
Confusion matrix: 
 [[ 82 337]
 [  6 172]]
Accuracy:  0.425460636515912